## Imports

In [27]:
#export
import pandas as pd
import torch
from torch import tensor
from sklearn.metrics import f1_score, precision_score, recall_score
from IPython.display import Markdown, clear_output
import ipywidgets as widgets

## Get

In [10]:
#export
def get_metrics(god):
    metrics = {'train': {}, 'val': {}}
    multi_epoch_metrics = {}
    for metric_config in god.config['training']['metrics']:
        for phase in ('train', 'val'):
            if metric_config['phase'] in (phase, 'both'):
                metric_name, metric_dict = get_metric(god, metric_config, phase)
                metrics[phase][metric_name] = metric_dict
                if metric_dict['get_episode_average']:
                    multi_epoch_metrics[metric_name] = metric_dict
    return metrics, multi_epoch_metrics

In [11]:
#export
def get_episode_metrics(god):
    episode_metrics = god.metrics_results['episodes'][god.state['episode_nr']]
    grouped_metrics_dict = {}
    for phase in god.metrics.keys():
        for metric_name in god.metrics[phase]:
            grouped_metric = {}
            for epoch_nr, epoch_dict in episode_metrics['epochs'].items():
                if phase in epoch_dict:
                    grouped_metric[epoch_nr] = epoch_dict[phase][metric_name]
                    
            grouped_metrics_dict[metric_name] = grouped_metric
     
    return grouped_metrics_dict

In [12]:
#export
# TODO adjust so it doesn't need every metric to be calculated every epoch
def get_episode_average(god):
    multi_episode_results = {}
    for metric_name, metric_dict in god.multi_epoch_metrics.items():
        episode_metric_results = []
        for episode_result_dict in  god.metrics_results['episodes'].values():
            epoch_metric_results = [epoch_result_dict[metric_dict['phase']][metric_name] for epoch_result_dict in episode_result_dict['epochs'].values()]
            episode_metric_results.append(epoch_metric_results)

        metric_results_tensor = tensor(episode_metric_results)
        epoch_means = metric_results_tensor.mean(dim=0)

        best_epoch_mean_idx = epoch_means.argmax()
        best_epoch_mean_result = epoch_means[best_epoch_mean_idx]
        best_epoch_mean_values = metric_results_tensor[:,best_epoch_mean_idx]
        best_epoch_mean_std = best_epoch_mean_values.std()

        multi_episode_results[metric_name] = {
            'best_epoch_mean': best_epoch_mean_result.item(),
            'best_epoch_mean_nr': best_epoch_mean_idx.item() + 1,
            'best_epoch_mean_std': best_epoch_mean_std.item(),
            'best_epoch_mean_values': best_epoch_mean_values.tolist()
        }
        
    return multi_episode_results

## Get Util

In [13]:
#export
def get_metric(god, metric_config, phase):    
    metric_name = f"{phase}_{metric_config['name']}"
    metric_object = get_metric_object(god, metric_config)
    get_episode_average = metric_config.get('get_episode_average', {}).get('phase', None) in {phase, 'both'}
    
    return metric_name, {'phase': phase, 'object': metric_object, 'get_episode_average': get_episode_average}


def get_metric_object(god, metric_config):
    if metric_config['name'] == 'loss':
        return LossMetric(god, metric_config['settings'])
    
    elif metric_config['name'] == 'accuracy':
        return AccuracyMetric(god, metric_config['settings'])
    
    elif metric_config['name'] == 'f1_score':
        return F1AccRecMetric(god, metric_config['settings'], f1_score)
    
    elif metric_config['name'] == 'precision':
        return F1AccRecMetric(god, metric_config['settings'], precision_score)
    
    elif metric_config['name'] == 'recall':
        return F1AccRecMetric(god, metric_config['settings'], recall_score)  
    
    else:
        raise Exception(f"Metric with name '{metric_config['name']}' not supported.'")

## Output Metrics

In [14]:
#export
def show_table_phase_metrics(god):
    formatted_results = []
    for epoch_nr, epoch_result in god.metrics_results['episodes'][god.state['episode_nr']]['epochs'].items():
        for phase, phase_result in epoch_result.items(): 
            res = {}
            res['epoch_nr'] = epoch_nr
            res['phase'] = phase
            for metric_name, metric_result in phase_result.items():
                # todo add table precision constant
                rounded_metric_result = round(metric_result, 5) if type(metric_result) != list else [round(r, 5) for r in metric_result]
                res[metric_name.split('_', 1)[1]] = rounded_metric_result
            formatted_results.append(res)

    df = pd.DataFrame(formatted_results)
    df_val = df.loc[df['phase'] == 'val']
    df_train = df.loc[df['phase'] == 'train']

    with god.state['table_output']:
        god.state['table_output'].clear_output(wait=True)
        display(Markdown(df_train.to_markdown(index=False)))
        print()
        display(Markdown(df_val.to_markdown(index=False)))
        print()
    

In [15]:
#export
def print_phase_metrics(god):
    print_precision = get_print_precision(god)

    print(f"=== Epoch: {god.state['epoch_nr']}, Phase: {god.state['phase']} ===")
    phase_metrics = god.metrics_results['episodes'][god.state['episode_nr']]['epochs'][god.state['epoch_nr']][god.state['phase']]
    for name,result_dict in phase_metrics.items():
        print(f"{name}: {round(result_dict, print_precision)}")
    print()

In [16]:
#export
def print_multi_episodes_metrics(god):
    print_precision = get_print_precision(god)
    
    print(f"Multi Episode Results at Episode {god.state['episode_nr']}:")
    for metric_name, multi_episode_result in god.metrics_results['multi_episode_results'].items():
        print()
        print(f'  {metric_name}:')
        for stat_name,stat_value in multi_episode_result.items():
            stat_value = round(stat_value, print_precision) if type(stat_value) != list else [round(x, print_precision) for x in stat_value]
            print(f"    {stat_name}: {stat_value}")

In [17]:
#export
def get_print_precision(god):
    return god.config['constants']['print_precision'] if 'print_precision' in god.config['constants'] else 6

In [18]:
#WIP
def print_best_mean(val_acc_list):
    best_epoch_acc_mean = tensor(val_acc_list).mean(dim=0).max().item()
    best_epoch_nr = tensor(val_acc_list).mean(dim=0).argmax().item()
    best_epoch_std = tensor(val_acc_list)[:, best_epoch_nr].std().item()
    print(f'Best mean accuracy of {round(best_epoch_acc_mean, 4)} at epoch {best_epoch_nr} with n={len(val_acc_list)} (std: {round(best_epoch_std, 6)})')   

In [19]:
#WIP
def WIP_get_ensemble_accs(config, model_ensemble):
    ensemble_pred, y = ensemble_prediction(config, model_ensemble, mode=False)
    mean_acc = (ensemble_pred == y).float().mean().item()

    ensemble_pred, y = ensemble_prediction(config, model_ensemble, mode=True)
    mode_acc = (ensemble_pred == y).float().mean().item()

    return mean_acc, mode_acc

## Metric Updates

In [28]:
#export
class MetricUpdates:

    def update_before_episode_metrics(god):
        if god.config['training']['output'].get('show_metrics', {}).get('print', {}).get('episode', False):
            print(f"\n ===== Episode Nr {god.state['episode_nr']} =====")
            
        if not 'episodes' in god.metrics_results:
            god.metrics_results['episodes'] = {}
        if not 'multi_episode_results' in god.metrics_results and god.multi_epoch_metrics:
            god.metrics_results['multi_episode_results'] = {}
            for metric_name in god.multi_epoch_metrics:
                god.metrics_results['multi_episode_results'][metric_name] = {'epoch_means': []}
        
        god.metrics_results['episodes'][god.state['episode_nr']] = {'epochs': {}}

    def update_before_epoch_metrics(god):
        # TODO: TEMPORARY POSITION UNTIL PROGRESS BAR HAS OWN OUTPUT
        if god.state['epoch_nr'] == 1:
            if 'table_output' in god.state:
                god.state['table_output'].clear_output(wait=True)
            god.state['table_output'] = widgets.Output()
            display(god.state['table_output'])
        
        god.metrics_results['episodes'][god.state['episode_nr']]['epochs'][god.state['epoch_nr']] = {}

    def update_before_phase_metrics(god):
        god.metrics_results['episodes'][god.state['episode_nr']]['epochs'][god.state['epoch_nr']][god.state['phase']] = {}
        for metric_dict in god.metrics[god.state['phase']].values():
            metric_dict['object'].before_phase()

    def update_before_batch_metrics(god):
        for metric_dict in god.metrics[god.state['phase']].values():
            metric_dict['object'].before_batch()

    def update_after_batch_metrics(god):
        for metric_dict in god.metrics[god.state['phase']].values():
            metric_dict['object'].after_batch()

    def update_after_phase_metrics(god):
        phase_results = god.metrics_results['episodes'][god.state['episode_nr']]['epochs'][god.state['epoch_nr']][god.state['phase']]
        for metric_name, metric_dict in god.metrics[god.state['phase']].items():
            metric_dict['object'].after_phase()
            phase_results[metric_name] = metric_dict['object'].epoch_result['value']
                
        # TODO temp solution, make it a callback
        show_metrics = god.config['training']['output'].get('show_metrics', {})

        if show_metrics.get('print', {}).get('epoch', False):
            print_phase_metrics(god)
        
        if show_metrics.get('table', False):
            show_table_phase_metrics(god)
        
        if show_metrics.get('plot', False):
            raise Exception("show_metrics for 'plot' not implemented yet")
                
    def update_after_epoch_metrics(god):
        pass  

    def update_after_episode_metrics(god):
        store_episode_metrics(god)
        store_multi_episode_metrics(god)
        if god.config['training']['output'].get('show_metrics', {}).get('print', {}).get('episode', False):
            if len(god.multi_epoch_metrics) > 0:
                print_multi_episodes_metrics(god)

In [21]:
#export
def store_episode_metrics(god):
    god.metrics_results['episodes'][god.state['episode_nr']]['episode_metrics'] = get_episode_metrics(god)


def store_multi_episode_metrics(god):
    if god.multi_epoch_metrics:
        god.metrics_results['multi_episode_results'] = get_episode_average(god)
    # TODO add more multi episode metrics

## Metric

In [22]:
#export
class Metric:
    def __init__(self, god, settings):
        self.god = god
        self.settings = settings
    
    def before_epoch(self):
        pass
    
    def after_epoch(self):
        pass
    
    def before_phase(self):
        pass
    
    def after_phase(self):
        pass
    
    def before_batch(self):
        pass
    
    def after_batch(self):
        pass

In [23]:
#export
class LossMetric(Metric):
    
    def before_phase(self):
        self.losses = []
        self.ns = []
        self.epoch_result = None
        
    def after_batch(self):
        pred, yb = self.god.state['pred'], self.god.state['yb']
        loss = self.god.criterion(pred, yb)
        self.losses.append(loss.item())
        self.ns.append(yb.size(0))

    def after_phase(self):
        self.epoch_result = {
            'value': ((tensor(self.losses) * tensor(self.ns)).sum()  / tensor(self.ns).sum()).item(),
            # 'values': self.losses,
            # 'ns': self.ns,
            # 'n': tensor(self.ns).sum().item()
        }

In [24]:
#export
class AccuracyMetric(Metric):

    def before_phase(self):
        self.corrects = []
        self.ns = []
        self.values = []
        self.epoch_result = None
    
    def after_batch(self):
        pred_class, yb = self.god.state['pred_class'], self.god.state['yb']
        self.corrects.append((pred_class == yb).float().sum().item())
        self.ns.append(yb.size(0))
        self.values.append((pred_class == yb).float().mean().item())
        
    def after_phase(self):
        self.epoch_result = {
            'value':  sum(self.corrects) / sum(self.ns),
            # 'values': self.values,
            # 'corrects': self.corrects,
            # 'ns': self.ns
        }

In [25]:
#export
class F1AccRecMetric(Metric):
    
    def __init__(self, god, settings, func):
        self.god = god
        self.settings = settings
        self.func = func
        self.average = self.settings.get('average', 'binary')
        self.labels = self.settings.get('labels', None)
        
    def before_phase(self):
        self.epoch_result = None

    def after_phase(self):
        value = self.func(self.god.state['all_yb'], self.god.state['all_pred_class'], average=self.average, labels=self.labels)
        self.epoch_result = {'value':  value}

## Test+Export

In [26]:
!python _notebook2script.py 08_Metrics.ipynb

Converted 08_Metrics.ipynb to exp/Metrics.py
